In [ ]:
%%capture [--no-stdout]
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)
import itables.options as opt

opt.classes="display nowrap compact"
opt.buttons=["copyHtml5", "csvHtml5", "excelHtml5"]
opt.maxBytes=0
%env FHIR_BASE= https://google-fhir.fhir-aggregator.org

!fq vocabulary vocabulary.tsv --fhir-base-url $FHIR_BASE
df = pd.read_csv('vocabulary.tsv', sep='\t').fillna('')


# Vocabulary filtering examples

It's easy enough to get a straightforward table response - just utilize Pandas to filter appropriately to your interest. Some examples below:

# Exploring a study
Finding the vocabulary specific study is easy - filter according to the identifier in the `research_study_identifiers` column.

In [ ]:
scratch_df = df[df['research_study_identifiers'].str.contains('1KG')]
scratch_df

research_study_identifiers                         path  \
3391                               1KG  DocumentReference.extension   
3724                       Cellosaurus            Patient.extension   
3780   BEATAML1.0-CRENOLANIB,phs001628  DocumentReference.extension   
3785   BEATAML1.0-CRENOLANIB,phs001628            Patient.extension   
4130                   FM-AD,phs001179  DocumentReference.extension   
...                                ...                          ...   
20785                        TCGA-HNSC  DocumentReference.extension   
20798                        TCGA-HNSC            Patient.extension   
21148                        TCGA-KIRC  DocumentReference.extension   
21158                        TCGA-KIRC            Patient.extension   
21733                        HCMI-CMDC  DocumentReference.extension   

                                           extension_url      low  \
3391   https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...  36047.0   
3724   http://hl7.org/fhir/SearchParameter/patient-ex...      1.0   
3780   https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...   4220.0   
3785   http://hl7.org/fhir/SearchParameter/patient-ex...     24.0   
4130   https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...    867.0   
...                                                  ...      ...   
20785  https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...    229.0   
20798  http://hl7.org/fhir/SearchParameter/patient-ex...     19.0   
21148  https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...    229.0   
21158  http://hl7.org/fhir/SearchParameter/patient-ex...     26.0   
21733  https://nih-ncpi.github.io/ncpi-fhir-ig-2/Stru...    229.0   

                 high url  
3391    16099441977.0      
3724             94.0      
3780    45589228155.0      
3785             87.0      
4130        1935511.0      
...               ...  ..  
20785  578035949313.0      
20798            89.0      
21148  481092668932.0      
21158            89.0      
21733  484490562985.0      

[140 rows x 6 columns]

# Exploring available conditions
Or query the vocab table for the available condition codes of the various patients in each research study.

In [ ]:
scratch_df = df[df['path'].str.contains('Condition.code')]
scratch_df

# Something more elaborate
In lieu of pandas filtering, we can use curl to perform more elaborate filtering. In this case we're searching for a specific condition, Neuroblastoma. Here's what each part of the below code is doing:
1. FHIR Query: The curl command sends the FHIR query to the server.
2. Data Extraction and Formatting: jq extracts the system, code, and display information from the coding elements within each Condition resource and formats them as TSV.
3. Sorting: The output is piped to sort to arrange the entries alphabetically.
4. Deduplication: The -u option in sort removes any duplicate entries, leaving only unique combinations of system, code, and display.
5. Output: The final result is a sorted and deduplicated list of coding information for conditions starting with "Adenocarcinoma," presented as TSV.

In [ ]:
!curl -s $FHIR_BASE'/Condition?code:text=Neuroblastoma&_count=1000&_total=accurate&_elements=subject,extension,code'  | jq -rc ' .entry[] | .resource | .code.coding[] | [.system, .code, .display] | @tsv' | sort | uniq -c
#| jq -rc '.entry[] | .resource | [.subject.reference, (.extension[] | .valueReference.reference)] '